In [1]:
# This code computes the coefficients.

import sympy as sp

# Set the number of segments (N must be at least 4)
N = 3

# Create lists of unknown coefficients for each segment.
# For segment i (i=0,...,N-1) we have cubic polynomial coefficients: a_i, b_i, c_i, d_i.
a = [sp.symbols(f'a{i}', real=True) for i in range(N)]
b = [sp.symbols(f'b{i}', real=True) for i in range(N)]
c = [sp.symbols(f'c{i}', real=True) for i in range(N)]
d = [sp.symbols(f'd{i}', real=True) for i in range(N)]

# Build a list "unknowns" in the order: a0, b0, c0, d0, a1, b1, c1, d1, ..., a{N-1}, b{N-1}, c{N-1}, d{N-1}
unknowns = []
for i in range(N):
    unknowns.extend([a[i], b[i], c[i], d[i]])

# Define symbols for the boundary conditions.
P0, V0, A0 = sp.symbols('P0 V0 A0', real=True)   # initial state: position, velocity, acceleration
Pf, Vf, Af = sp.symbols('Pf Vf Af', real=True)     # final state

# Define time durations for each segment.
# For segment i, the time duration is T_i.
T = [sp.symbols(f'T{i}', positive=True, real=True) for i in range(N)]

# Build the list of equations.
eqs = []

# --- Initial conditions for segment 0 at t = 0 ---
eqs.append(sp.Eq(d[0], P0))      # p0(0)= d0 = P0
eqs.append(sp.Eq(c[0], V0))      # p0'(0)= c0 = V0
eqs.append(sp.Eq(2*b[0], A0))    # p0''(0)= 2*b0 = A0   => b0 = A0/2

# --- Continuity conditions for segments 0 to N-2 ---
for i in range(N - 1):
    # Position continuity: p_i(T_i)= a_i*T_i^3 + b_i*T_i^2 + c_i*T_i + d_i equals the start of segment i+1, i.e. d_{i+1}.
    eqs.append(sp.Eq(a[i]*T[i]**3 + b[i]*T[i]**2 + c[i]*T[i] + d[i], d[i+1]))
    # Velocity continuity: p_i'(T_i)= 3*a_i*T_i^2 + 2*b_i*T[i] + c[i] equals c_{i+1}.
    eqs.append(sp.Eq(3*a[i]*T[i]**2 + 2*b[i]*T[i] + c[i], c[i+1]))
    # Acceleration continuity: p_i''(T_i)= 6*a[i]*T[i] + 2*b[i] equals 2*b[i+1].
    eqs.append(sp.Eq(6*a[i]*T[i] + 2*b[i], 2*b[i+1]))

# --- Final conditions for segment N-1 at t = T_{N-1} ---
eqs.append(sp.Eq(a[N-1]*T[N-1]**3 + b[N-1]*T[N-1]**2 + c[N-1]*T[N-1] + d[N-1], Pf))
eqs.append(sp.Eq(3*a[N-1]*T[N-1]**2 + 2*b[N-1]*T[N-1] + c[N-1], Vf))
eqs.append(sp.Eq(6*a[N-1]*T[N-1] + 2*b[N-1], Af))

print("Number of unknowns:", len(unknowns))
print("Number of equations:", len(eqs))  # Should be 3N+3

# ----- (Optional) Solve the system symbolically -----
solution = sp.linsolve(eqs, unknowns)
if not solution:
    print("No solution found!")
else:
    sol_tuple = list(solution)[0]  # This is a tuple of solutions (expressed in terms of free parameters)
    print("\n// C++ Code for the coefficients:")
    for var, expr in zip(unknowns, sol_tuple):
        print("double {} = {};".format(var, sp.ccode(expr)))

# =======================
# 2. Compute Control Points
# =======================
#
# For each segment i (with duration T_i) the cubic polynomial is:
#   p_i(t)= a_i*t^3 + b_i*t^2 + c_i*t + d_i, for t in [0, T_i].
#
# We define the position control points as:
#   CP0 = p_i(0) = d_i,
#   CP1 = (c_i*T_i + 3*d_i) / 3,
#   CP2 = (b_i*T_i**2 + 2*c_i*T_i + 3*d_i) / 3,
#   CP3 = p_i(T_i) = a_i*T_i^3 + b_i*T_i^2 + c_i*T_i + d_i.
#
# And for the derivative curves (defined on the normalized parameter u=t/T_i):
#   Velocity: v_i(t)= 3*a_i*t^2+ 2*b_i*t+c_i.
#     -> Define V0 = c_i, V1 = c_i + (2*b_i*T_i)/2, V2 = c_i+2*b_i*T_i+ 3*a_i*T_i^2.
#   Acceleration: a_i(t)= 6*a_i*t+2*b_i.
#     -> A0 = 2*b_i, A1 = 2*b_i+6*a_i*T_i.
#   Jerk: j_i(t)= 6*a_i (constant).
#     -> J0 = 6*a_i.
#
# Define helper functions:

def position_control_points(a_coef, b_coef, c_coef, d_coef, T_val):
    CP0 = d_coef
    CP1 = (c_coef*T_val + 3*d_coef) / 3
    CP2 = (b_coef*T_val**2 + 2*c_coef*T_val + 3*d_coef) / 3
    CP3 = sp.simplify(a_coef*T_val**3 + b_coef*T_val**2 + c_coef*T_val + d_coef)
    return (CP0, CP1, CP2, CP3)

def velocity_control_points(a_coef, b_coef, c_coef, d_coef, T_val):
    # Velocity: v(t)= 3*a*t^2+2*b*t+c.
    v0 = sp.simplify(c_coef)  # at t=0
    v_mid = sp.simplify(3*a_coef*(T_val/2)**2 + 2*b_coef*(T_val/2) + c_coef)
    vT = sp.simplify(3*a_coef*T_val**2 + 2*b_coef*T_val + c_coef)
    return (v0, v_mid, vT)

def acceleration_control_points(a_coef, b_coef, c_coef, d_coef, T_val):
    # Acceleration: a(t)= 6*a*t+2*b.
    a0_val = sp.simplify(2*b_coef)  # at t=0
    aT_val = sp.simplify(6*a_coef*T_val + 2*b_coef)
    return (a0_val, aT_val)

def jerk_control_point(a_coef, b_coef, c_coef, d_coef, T_val):
    # Jerk: j(t)= 6*a, constant.
    return sp.simplify(6*a_coef)

# Substitute the solution into the unknowns:
subs_dict = dict(zip(unknowns, sol_tuple))
# For convenience, define lists for each segment’s coefficients:
a_sol = [subs_dict[a[i]] for i in range(N)]
b_sol = [subs_dict[b[i]] for i in range(N)]
c_sol = [subs_dict[c[i]] for i in range(N)]
d_sol = [subs_dict[d[i]] for i in range(N)]

# Compute control points for each segment.
CP = []      # position control points for each segment
VelCP = []   # velocity control points (as a triple) for each segment
AccelCP = [] # acceleration control points (as a pair) for each segment
JerkCP = []  # jerk control point for each segment

for i in range(N):
    CP.append(position_control_points(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))
    VelCP.append(velocity_control_points(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))
    AccelCP.append(acceleration_control_points(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))
    JerkCP.append(jerk_control_point(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))

# =======================
# 3. Print the results in C++ code format
# =======================
print("\n// POSITION CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    CPi = CP[i]
    for j, cp in enumerate(CPi):
        print("double CP{0}_{1} = {2};".format(j, i, sp.ccode(cp)))

print("\n// VELOCITY CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    VCPi = VelCP[i]
    for j, cp in enumerate(VCPi):
        print("double vCP{0}_{1} = {2};".format(j, i, sp.ccode(cp)))

print("\n// ACCELERATION CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    ACPi = AccelCP[i]
    for j, cp in enumerate(ACPi):
        print("double aCP{0}_{1} = {2};".format(j, i, sp.ccode(cp)))

print("\n// JERK CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    print("double jCP0_{0} = {1};".format(i, sp.ccode(JerkCP[i])))


Number of unknowns: 12
Number of equations: 12

// C++ Code for the coefficients:
double a0 = (-3*A0*pow(T0, 2) - 4*A0*T0*T1 - 2*A0*T0*T2 - A0*pow(T1, 2) - A0*T1*T2 + Af*T1*T2 + Af*pow(T2, 2) - 6*P0 + 6*Pf - 6*T0*V0 - 4*T1*V0 - 2*T1*Vf - 2*T2*V0 - 4*T2*Vf)/(6*pow(T0, 3) + 12*pow(T0, 2)*T1 + 6*pow(T0, 2)*T2 + 6*T0*pow(T1, 2) + 6*T0*T1*T2);
double b0 = (1.0/2.0)*A0;
double c0 = V0;
double d0 = P0;
double a1 = (A0*pow(T0, 3) + 4*A0*pow(T0, 2)*T1 + 2*A0*pow(T0, 2)*T2 + 3*A0*T0*pow(T1, 2) + 3*A0*T0*T1*T2 + A0*T0*pow(T2, 2) - Af*pow(T0, 2)*T2 - 3*Af*T0*T1*T2 - 2*Af*T0*pow(T2, 2) - 3*Af*pow(T1, 2)*T2 - 4*Af*T1*pow(T2, 2) - Af*pow(T2, 3) + 6*P0*T0 + 12*P0*T1 + 6*P0*T2 - 6*Pf*T0 - 12*Pf*T1 - 6*Pf*T2 + 4*pow(T0, 2)*V0 + 2*pow(T0, 2)*Vf + 12*T0*T1*V0 + 6*T0*T1*Vf + 6*T0*T2*V0 + 6*T0*T2*Vf + 6*pow(T1, 2)*V0 + 6*pow(T1, 2)*Vf + 6*T1*T2*V0 + 12*T1*T2*Vf + 2*pow(T2, 2)*V0 + 4*pow(T2, 2)*Vf)/(6*pow(T0, 2)*pow(T1, 2) + 6*pow(T0, 2)*T1*T2 + 12*T0*pow(T1, 3) + 18*T0*pow(T1, 2)*T2 + 6*T0*T1*pow(T2, 2) + 6

In [21]:
# This code checks if the control points depend on the free parameters.
import sympy as sp

# -------------------------------------------------------------------------
# This cell assumes that you have already computed a solution for your spline
# equations and built a substitution dictionary (subs_dict) with Af=0 and Vf=0.
# It also assumes that you have computed the control points for each segment.
#
# For example, the previous cell defined:
#   N: the number of segments.
#   For each segment i, the coefficients a_i, b_i, c_i, d_i,
#   and time duration T_i.
#
# Then using functions position_control_points, velocity_control_points,
# acceleration_control_points, and jerk_control_point, you obtained:
#
#    CP[i]      : a tuple (CP0, CP1, CP2, CP3) for segment i (position)
#    VelCP[i]   : a tuple (vCP0, vCP1, vCP2) for segment i (velocity)
#    AccelCP[i] : a tuple (aCP0, aCP1) for segment i (acceleration)
#    JerkCP[i]  : a single expression for segment i (jerk)
#
# In addition, the free parameters in the solution (those symbols not in 
# {P0, V0, A0, Pf, Vf, Af} ∪ {T0,...,T_{N-1}}) have been computed.
#
# Below, we produce two separate outputs:
#  (1) The control point C++ declarations.
#  (2) A report (per control point) of which free parameters the expression depends on.
# -------------------------------------------------------------------------

# For demonstration purposes, let’s assume these lists are already computed.
# They should be computed similarly to the code in your previous cell.
# (For example, see the code provided in earlier answers.)
# Here we assume that:
#    CP, VelCP, AccelCP, JerkCP are lists indexed by segment (0 ... N-1).
# And that free_params is a set of free symbols.
# For example, after:
#    free_syms = set()
#    for expr in sol_tuple:
#         free_syms |= expr.free_symbols
#    free_params = free_syms - ({P0, V0, A0, Pf, Vf, Af} | set(T))
#
# The following functions are defined to check dependency:

def depends_on_free_params(expr, free_params):
    """
    Return the subset of free_params that occur in expr.
    """
    return list(expr.free_symbols & free_params)

# ---- Functions to print control points code (without dependency info) ----

def print_cp_code(cp_tuple, seg, cp_type):
    """
    Prints C++ declaration code for each control point in cp_tuple for segment 'seg'
    of type cp_type (e.g. "Position", "Velocity", "Acceleration").
    """
    for i, expr in enumerate(cp_tuple):
        print("double {}CP{}_{} = {};".format(cp_type[0].lower(), i, seg, sp.ccode(expr)))

def print_single_cp_code(expr, seg, cp_type):
    print("double {}CP0_{} = {};".format(cp_type[0].lower(), seg, sp.ccode(expr)))

# ---- Functions to print dependency info ----

def print_cp_dependency(cp_tuple, seg, cp_type, free_params):
    for i, expr in enumerate(cp_tuple):
        deps = depends_on_free_params(expr, free_params)
        if deps:
            dep_str = "depends on " + ", ".join(sorted([sp.ccode(s) for s in deps]))
        else:
            dep_str = "independent of free parameters"
        print("// {}CP{}_{}: {}".format(cp_type, i, seg, dep_str))

def print_single_cp_dependency(expr, seg, cp_type, free_params):
    deps = depends_on_free_params(expr, free_params)
    if deps:
        dep_str = "depends on " + ", ".join(sorted([sp.ccode(s) for s in deps]))
    else:
        dep_str = "independent of free parameters"
    print("// {}CP0_{}: {}".format(cp_type, seg, dep_str))

# -------------------------
# The following variables are assumed to be defined:
#   N, CP, VelCP, AccelCP, JerkCP, free_params.
# For example, from previous computations:
#
# N = <number of segments>
# CP[i] = position control points tuple for segment i,
# VelCP[i] = velocity control points tuple for segment i,
# AccelCP[i] = acceleration control points tuple for segment i,
# JerkCP[i] = jerk control point for segment i.
#
# And free_params is computed as:
#   known = {P0, V0, A0, Pf, Vf, Af} | set([T0, T1, ..., T_{N-1}])
#   free_syms = set()
#   for expr in sol_tuple: free_syms |= expr.free_symbols
#   free_params = free_syms - known
#
# For this code to run, these must be defined.
# -------------------------

# For demonstration, if you haven't computed free_params, you can compute it here:
known_syms = {P0, V0, A0, Pf, Vf, Af}
for t_sym in T:
    known_syms.add(t_sym)
free_syms = set()
# Assume sol_tuple is the tuple of solutions from your previous linsolve.
# (If not defined, you must run that cell first.)
# For each expression in the solution:
for expr in sol_tuple:
    free_syms |= expr.free_symbols
free_params = free_syms - known_syms

# -------------------------
# 4. Print Separate Outputs
# -------------------------

print("\n// *** CONTROL POINTS CODE OUTPUT ***")
# Print Position Control Points:
print("\n// POSITION CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    print_cp_code(CP[i], str(i), "Position")
    
# Print Velocity Control Points:
print("\n// VELOCITY CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    print_cp_code(VelCP[i], str(i), "Velocity")
    
# Print Acceleration Control Points:
print("\n// ACCELERATION CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    print_cp_code(AccelCP[i], str(i), "Acceleration")
    
# Print Jerk Control Points:
print("\n// JERK CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    print_single_cp_code(JerkCP[i], str(i), "Jerk")

print("\n// *** FREE PARAMETER DEPENDENCY INFORMATION ***")
# Print dependency info for Position CPs:
print("\n// POSITION CONTROL POINTS DEPENDENCY:")
for i in range(N):
    print(f"// Segment {i}:")
    print_cp_dependency(CP[i], str(i), "Position", free_params)
    
# Print dependency info for Velocity CPs:
print("\n// VELOCITY CONTROL POINTS DEPENDENCY:")
for i in range(N):
    print(f"// Segment {i}:")
    print_cp_dependency(VelCP[i], str(i), "Velocity", free_params)
    
# Print dependency info for Acceleration CPs:
print("\n// ACCELERATION CONTROL POINTS DEPENDENCY:")
for i in range(N):
    print(f"// Segment {i}:")
    print_cp_dependency(AccelCP[i], str(i), "Acceleration", free_params)
    
# Print dependency info for Jerk CPs:
print("\n// JERK CONTROL POINTS DEPENDENCY:")
for i in range(N):
    print(f"// Segment {i}:")
    print_single_cp_dependency(JerkCP[i], str(i), "Jerk", free_params)



// *** CONTROL POINTS CODE OUTPUT ***

// POSITION CONTROL POINTS:
// Segment 0:
double pCP0_0 = P0;
double pCP1_0 = P0 + (1.0/3.0)*T0*V0;
double pCP2_0 = (1.0/6.0)*A0*pow(T0, 2) + P0 + (2.0/3.0)*T0*V0;
double pCP3_0 = (1.0/6.0)*(pow(T0, 2)*(T3*T4*(-3*A0*pow(T0, 2)*T3*T4*pow(T5, 2) - 4*A0*T0*T1*T3*T4*pow(T5, 2) - 2*A0*T0*T2*T3*T4*pow(T5, 2) - A0*pow(T1, 2)*T3*T4*pow(T5, 2) - A0*T1*T2*T3*T4*pow(T5, 2) - 8*Af*T1*T2*T3*T4*pow(T5, 2) - 6*Af*T1*T2*T3*pow(T5, 3) - 6*Af*T1*T2*pow(T4, 2)*pow(T5, 2) - 6*Af*T1*T2*T4*pow(T5, 3) - 4*Af*T1*pow(T3, 2)*T4*pow(T5, 2) - 3*Af*T1*pow(T3, 2)*pow(T5, 3) - 4*Af*T1*T3*pow(T4, 2)*pow(T5, 2) - 4*Af*T1*T3*T4*pow(T5, 3) - 8*Af*pow(T2, 2)*T3*T4*pow(T5, 2) - 6*Af*pow(T2, 2)*T3*pow(T5, 3) - 6*Af*pow(T2, 2)*pow(T4, 2)*pow(T5, 2) - 6*Af*pow(T2, 2)*T4*pow(T5, 3) - 8*Af*T2*pow(T3, 2)*T4*pow(T5, 2) - 6*Af*T2*pow(T3, 2)*pow(T5, 3) - 8*Af*T2*T3*pow(T4, 2)*pow(T5, 2) - 8*Af*T2*T3*T4*pow(T5, 3) - 6*P0*T3*T4*pow(T5, 2) - 24*Pf*T1*T2*T3*T4 - 36*Pf*T1*T2*T3*T5 - 18*Pf*T1*T2*p